En esta notebook empezamos descargando el historico de mails diarios de compromiso. Despues generamos una tabla con el historico en sql. La idea es despues crear un proceso en sql que le inserte a la tabla de compromisos historica, el compromiso diario.

In [1]:
from utils.functions_ia import *

In [3]:
# Aca se descargan los archivos de los mails que contengan como asunto los reportes de compromiso
subject = "Compromisos: reporte"
sent_from = "datos.mopc@gmail.com"
download_path ="excels_compromiso/"

download_email_attachments(subject,sent_from,download_path)


excels_compromiso//compromisos_20210408.xlsx
excels_compromiso//crudo_compromisos_20210408.xlsx
excels_compromiso//compromisos_20210408.xlsx
excels_compromiso//crudo_compromisos_20210408.xlsx
excels_compromiso//compromisos_20210409.xlsx
excels_compromiso//crudo_compromisos_20210409.xlsx
excels_compromiso//compromisos_20210409.xlsx
excels_compromiso//crudo_compromisos_20210409.xlsx
excels_compromiso//compromisos_20210409.xlsx
excels_compromiso//crudo_compromisos_20210409.xlsx
excels_compromiso//compromisos_20210409.xlsx
excels_compromiso//crudo_compromisos_20210409.xlsx
excels_compromiso//compromisos_20210410.xlsx
excels_compromiso//crudo_compromisos_20210410.xlsx
excels_compromiso//compromisos_20210411.xlsx
excels_compromiso//crudo_compromisos_20210411.xlsx
excels_compromiso//compromisos_20210412.xlsx
excels_compromiso//crudo_compromisos_20210412.xlsx
excels_compromiso//compromisos_20210413.xlsx
excels_compromiso//crudo_compromisos_20210413.xlsx
excels_compromiso//compromisos_20210414.

In [37]:
import os
import re 

#aca abrimos los archivos que guardamos
df = None

for i,file in enumerate(os.listdir('excels_compromiso/')):
    name = os.path.splitext(file)[0]
    if 'crudo' in name:
        if df is None:
            df = pd.read_excel(f'excels_compromiso/{file}')
        else: 
            print(len(df))
            df2 = pd.read_excel(f'excels_compromiso/{file}')
            print(file, len(df2))
            df = df.append(df2, ignore_index = True)
            print(len(df))
            

263
crudo_compromisos_20210409.xlsx 459
722
722
crudo_compromisos_20210410.xlsx 17
739
739
crudo_compromisos_20210411.xlsx 0
739
739
crudo_compromisos_20210412.xlsx 448
1187
1187
crudo_compromisos_20210413.xlsx 313
1500
1500
crudo_compromisos_20210414.xlsx 333
1833
1833
crudo_compromisos_20210415.xlsx 304
2137
2137
crudo_compromisos_20210416.xlsx 332
2469
2469
crudo_compromisos_20210417.xlsx 0
2469
2469
crudo_compromisos_20210418.xlsx 0
2469
2469
crudo_compromisos_20210419.xlsx 230
2699
2699
crudo_compromisos_20210420.xlsx 301
3000
3000
crudo_compromisos_20210421.xlsx 403
3403
3403
crudo_compromisos_20210422.xlsx 326
3729
3729
crudo_compromisos_20210423.xlsx 265
3994
3994
crudo_compromisos_20210424.xlsx 28
4022
4022
crudo_compromisos_20210425.xlsx 0
4022
4022
crudo_compromisos_20210426.xlsx 238
4260
4260
crudo_compromisos_20210427.xlsx 290
4550
4550
crudo_compromisos_20210428.xlsx 288
4838
4838
crudo_compromisos_20210429.xlsx 317
5155
5155
crudo_compromisos_20210430.xlsx 180
5335
5335


In [18]:
df.columns


Index(['IDCLIENTE', 'FECHA', 'FECHA DE PAGO', 'NOMBRE', 'IDMOROSO', 'FILTROA',
       'AMOUNT', 'PROVINCIA', 'IDOPERADOR', 'GRUPO', 'ESTADOINICIAL',
       'ESTADOFINAL', 'FASIGNACION', 'STATUS'],
      dtype='object')

In [38]:
#estasson modificaciones de fomratos

import datetime
print(type(df['FECHA']))
b = df[ df['FECHA'].dt.strftime('%Y-%d-%m') ==  '2021-06-05']
df['FECHA'] = df['FECHA'].dt.strftime('%Y-%d-%m %H:%M:%S')
df['FECHA DE PAGO'] = df['FECHA DE PAGO'].dt.strftime('%Y-%d-%m %H:%M:%S')
df['FASIGNACION'] = df['FASIGNACION'].dt.strftime('%Y-%d-%m %H:%M:%S')
df['IDCLIENTE'] = df['IDCLIENTE'].str.strip()
df['NOMBRE'] = df['NOMBRE'].str.strip()
df['IDMOROSO'] = df['IDMOROSO'].str.strip()
df['GRUPO'] = df['GRUPO'].fillna(0)

df.head(5)

<class 'pandas.core.series.Series'>


,IDCLIENTE,FECHA,FECHA DE PAGO,NOMBRE,IDMOROSO,FILTROA,AMOUNT,PROVINCIA,IDOPERADOR,GRUPO,ESTADOINICIAL,ESTADOFINAL,FASIGNACION,STATUS
0,BSR_TARDIA,2021-08-04 13:17:47,2021-10-04 00:00:00,ASIA T Y A SRL,CUIT30713476109,Campana Extrajud Emp,5770.00,C1407GPR,ORTELLADOG,0,SKIP LEVEL 2,COMPROMISO,2019-02-12 00:00:00,NEW
1,BSR_TARDIA,2021-08-04 14:09:58,2020-10-06 00:00:00,SHUN QI SA,CUIT30715094149,Campana Extrajud Emp,1303.05,B1832CEI,ALONSOA,0,NO UBICADO,COMPROMISO,2019-02-12 00:00:00,BROKEN
2,BSR_TARDIA,2021-08-04 14:09:58,2020-16-06 00:00:00,SHUN QI SA,CUIT30715094149,Campana Extrajud Emp,1303.05,B1832CEI,ALONSOA,0,NO UBICADO,COMPROMISO,2019-02-12 00:00:00,BROKEN
3,BSR_TARDIA,2021-08-04 14:09:58,2020-18-06 00:00:00,SHUN QI SA,CUIT30715094149,Campana Extrajud Emp,1303.05,B1832CEI,ALONSOA,0,NO UBICADO,COMPROMISO,2019-02-12 00:00:00,BROKEN
4,BSR_TARDIA,2021-08-04 14:09:58,2020-26-06 00:00:00,SHUN QI SA,CUIT30715094149,Campana Extrajud Emp,1303.05,B1832CEI,ALONSOA,0,NO UBICADO,COMPROMISO,2019-02-12 00:00:00,BROKEN


In [7]:
#esto es una conexion falsa a una base de datos. Lo hago para convertir el dataframe a formato de tabla sql
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
df.to_sql('Tabla',con=engine)

In [20]:
a = engine.execute("SELECT IDCLIENTE, FECHA, [FECHA DE PAGO], NOMBRE, IDMOROSO, FILTROA, AMOUNT, PROVINCIA, IDOPERADOR, GRUPO, ESTADOINICIAL, ESTADOFINAL, FASIGNACION, STATUS FROM Tabla").fetchall()
a

[('BSR_TARDIA', '2021-08-04 13:17:47', '2021-10-04 00:00:00', 'ASIA T Y A SRL', 'CUIT30713476109', 'Campana Extrajud Emp', 5770.0, 'C1407GPR            ', 'ORTELLADOG', '0', 'SKIP LEVEL 2        ', 'COMPROMISO          ', '2019-02-12 00:00:00', 'NEW'),
 ('BSR_TARDIA', '2021-08-04 14:09:58', '2020-10-06 00:00:00', 'SHUN QI SA', 'CUIT30715094149', 'Campana Extrajud Emp', 1303.05, 'B1832CEI            ', 'ALONSOA   ', '0', 'NO UBICADO          ', 'COMPROMISO          ', '2019-02-12 00:00:00', 'BROKEN'),
 ('BSR_TARDIA', '2021-08-04 14:09:58', '2020-16-06 00:00:00', 'SHUN QI SA', 'CUIT30715094149', 'Campana Extrajud Emp', 1303.05, 'B1832CEI            ', 'ALONSOA   ', '0', 'NO UBICADO          ', 'COMPROMISO          ', '2019-02-12 00:00:00', 'BROKEN'),
 ('BSR_TARDIA', '2021-08-04 14:09:58', '2020-18-06 00:00:00', 'SHUN QI SA', 'CUIT30715094149', 'Campana Extrajud Emp', 1303.05, 'B1832CEI            ', 'ALONSOA   ', '0', 'NO UBICADO          ', 'COMPROMISO          ', '2019-02-12 00:00:00',

In [51]:
#genero la conexion real con la db
cnxn = connect_database()

In [ ]:
#inserto la tabla que cree en la db
cursor = cnxn.cursor()
for i in a:
    print(i)
    query = f"INSERT INTO ACTIVE.DBO.COMPROMISOS_HISTORICO  ESTADOINICIAL, ESTADOFINAL, FASIGNACION, STATUS) VALUES {i}"
    cursor.execute(query)
    cnxn.commit()
    print('Insert Finish!')
cnxn.close()

In [ ]:
def insert_in_db():
    